In [2]:
import pandas as pd
import numpy as np
from faker import Faker
import random
import datetime
import boto3
import botocore
import pprint
import psycopg2
import configparser

import proyecto


### Iniciación de Variables

In [3]:

rdsIdentifier = 'banco-db' #nombre de la instancia
fake = Faker() 

## Proyecto estudiantes

#### Cargamos archivos de configuraciones

In [4]:
config = configparser.ConfigParser()
config.read('escec.cfg')

['escec.cfg']

#### Creamos instancia RDS

In [5]:
aws_conn = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                     aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                     region_name='us-east-1')

#### Verificación de instancias RDS

In [6]:
rdsInstanceIds = []

response = aws_conn.describe_db_instances()
for resp in response['DBInstances']:
    rdsInstanceIds.append(resp['DBInstanceIdentifier'])
    db_instance_status = resp['DBInstanceStatus']

print(f"DBInstanceIds {rdsInstanceIds}")

DBInstanceIds ['banco-db', 'dw-db']


#### Creación de servicios RDS

In [7]:
try:
    response = aws_conn.create_db_instance(
            AllocatedStorage=10,
            DBName=config.get('RDS', 'DB_NAME'),
            DBInstanceIdentifier=rdsIdentifier,
            DBInstanceClass="db.t3.micro",
            Engine="postgres",
            MasterUsername=config.get('RDS', 'DB_USER'),
            MasterUserPassword=config.get('RDS', 'DB_PASSWORD'),
            Port=int(config.get('RDS', 'DB_PORT')),
            VpcSecurityGroupIds=["sg-085316449dd4f00b4"],
            PubliclyAccessible=True
        )
    pprint.pprint (response)
except aws_conn.exceptions.DBInstanceAlreadyExistsFault as ex:
    print("La Instancia de Base de Datos ya Existe.")

La Instancia de Base de Datos ya Existe.


##### Obtenemos URL del Host

In [8]:
try:
     instances = aws_conn.describe_db_instances(DBInstanceIdentifier=rdsIdentifier)
     RDS_HOST = instances.get('DBInstances')[0].get('Endpoint').get('Address')
     print(RDS_HOST)
except Exception as ex:
     print("La instancia de base de datos no existe o aun no se ha terminado de crear.")
     print(ex)


banco-db.cfaunaiv0bu7.us-east-1.rds.amazonaws.com


In [8]:
import proyecto
print(proyecto.DDL_QUERY)


CREATE TABLE IF NOT EXISTS p_school(
    id_school INT PRIMARY KEY,
    school VARCHAR(4) UNIQUE
);

CREATE TABLE IF NOT EXISTS p_sex(
    id_sex INT PRIMARY KEY,
    sexo VARCHAR(4) UNIQUE
);

CREATE TABLE IF NOT EXISTS p_job(
    id_job INT PRIMARY KEY,
    job_name VARCHAR(15) UNIQUE,
    relationship VARCHAR(15) UNIQUE
);

CREATE TABLE IF NOT EXISTS p_age(
    id_age INT PRIMARY KEY,
    age VARCHAR(4) UNIQUE
);

CREATE TABLE IF NOT EXISTS p_guardian(
    id_guardian INT PRIMARY KEY,
    guardian VARCHAR(15) UNIQUE
);

CREATE TABLE IF NOT EXISTS p_internet(
    id_internet INT PRIMARY KEY,
    internet VARCHAR(4) UNIQUE
);

CREATE TABLE IF NOT EXISTS p_romantic(
    id_romantic INT PRIMARY KEY,
    romantic VARCHAR(4) UNIQUE
);

CREATE TABLE IF NOT EXISTS p_grades(
    id_grades BIGINT PRIMARY KEY,
    date_hour TIMESTAMP,
    g1 INT,
    g2 INT,
    g3 INT,
    signature INT,
    id_school INT,
    id_sex INT,
    id_job INT,
    id_age INT,
    id_guardian INT,
    id_internet I

### Conexión a la base de datos sql desde Python

In [10]:
import proyecto

try:
    db_conn = psycopg2.connect(
        database=config.get('RDS', 'DB_NAME'), 
        user=config.get('RDS', 'DB_USER'),
        password=config.get('RDS', 'DB_PASSWORD'), 
        host=RDS_HOST,
        port=config.get('RDS', 'DB_PORT')
    )

    cursor = db_conn.cursor()
    cursor.execute(proyecto.DDL_QUERY)
    db_conn.commit()
    print("Base de Datos Creada Exitosamente")
except Exception as ex:
    print("ERROR: Error al crear la base de datos.")
    print(ex)

Base de Datos Creada Exitosamente


##### Insertamos Datos en la Tabla de Transacciones

In [11]:
def insertDataToSQL(data_dict, table_name):
     postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""    
     df_data = pd.DataFrame.from_records(data_dict)
     try:
          response = df_data.to_sql(table_name, postgres_driver, index=False, if_exists='append')
          print(f'Se han insertado {response} nuevos registros.' )
     except Exception as ex:
          print(ex)

#### Insertar datos de sexo del estudiante

In [16]:
sexo = [
     {'id_sex': 1, 'sexo': 'M'}, 
     {'id_sex': 2, 'sexo': 'F'}
]

insertDataToSQL(sexo, 'p_sex')

(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "p_sex_pkey"
DETAIL:  Key (id_sex)=(1) already exists.

[SQL: INSERT INTO p_sex (id_sex, sexo) VALUES (%(id_sex)s, %(sexo)s)]
[parameters: ({'id_sex': 1, 'sexo': 'M'}, {'id_sex': 2, 'sexo': 'F'})]
(Background on this error at: https://sqlalche.me/e/14/gkpj)


#### Insertar datos de servicio de Internet

In [17]:
internet = [
     {'id_internet': 1, 'internet': 'yes'}, 
     {'id_internet': 2, 'internet': 'no'}
]

insertDataToSQL(internet, 'p_internet')

(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "p_internet_pkey"
DETAIL:  Key (id_internet)=(1) already exists.

[SQL: INSERT INTO p_internet (id_internet, internet) VALUES (%(id_internet)s, %(internet)s)]
[parameters: ({'id_internet': 1, 'internet': 'yes'}, {'id_internet': 2, 'internet': 'no'})]
(Background on this error at: https://sqlalche.me/e/14/gkpj)


#### Insertar el tipo de trabajo de los padres

In [21]:
t_trabajo = [
     {'id_job': 1, 'job_name': 'at_home','relationship':'father' }, 
     {'id_job': 2, 'job_name': 'health','relationship':'father'},
     {'id_job': 3, 'job_name': 'other','relationship':'father'},
     {'id_job': 4, 'job_name': 'services','relationship':'father'},
     {'id_job': 5, 'job_name': 'teacher','relationship':'father'},
     {'id_job': 6, 'job_name': 'at_home','relationship':'mother' }, 
     {'id_job': 7, 'job_name': 'health','relationship':'mother'},
     {'id_job': 8, 'job_name': 'other','relationship':'mother'},
     {'id_job': 9, 'job_name': 'services','relationship':'mother'},
     {'id_job': 10, 'job_name': 'teacher','relationship':'mother'}
]

insertDataToSQL(t_trabajo, 'p_job')

(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "p_job_pkey"
DETAIL:  Key (id_job)=(1) already exists.

[SQL: INSERT INTO p_job (id_job, job_name, relationship) VALUES (%(id_job)s, %(job_name)s, %(relationship)s)]
[parameters: ({'id_job': 1, 'job_name': 'at_home', 'relationship': 'father'}, {'id_job': 2, 'job_name': 'health', 'relationship': 'father'}, {'id_job': 3, 'job_name': 'other', 'relationship': 'father'}, {'id_job': 4, 'job_name': 'services', 'relationship': 'father'}, {'id_job': 5, 'job_name': 'teacher', 'relationship': 'father'}, {'id_job': 6, 'job_name': 'at_home', 'relationship': 'mother'}, {'id_job': 7, 'job_name': 'health', 'relationship': 'mother'}, {'id_job': 8, 'job_name': 'other', 'relationship': 'mother'}, {'id_job': 9, 'job_name': 'services', 'relationship': 'mother'}, {'id_job': 10, 'job_name': 'teacher', 'relationship': 'mother'})]
(Background on this error at: https://sqlalche.me/e/14/gkpj)


#### Insertar el tutor

In [22]:
guardian = [
     {'id_guardian': 1, 'guardian': 'father'}, 
     {'id_guardian': 2, 'guardian': 'mother'},
     {'id_guardian': 3, 'guardian': 'other'}
]

insertDataToSQL(guardian, 'p_guardian')

(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "p_guardian_pkey"
DETAIL:  Key (id_guardian)=(1) already exists.

[SQL: INSERT INTO p_guardian (id_guardian, guardian) VALUES (%(id_guardian)s, %(guardian)s)]
[parameters: ({'id_guardian': 1, 'guardian': 'father'}, {'id_guardian': 2, 'guardian': 'mother'}, {'id_guardian': 3, 'guardian': 'other'})]
(Background on this error at: https://sqlalche.me/e/14/gkpj)


#### Insertar si el alumno está en una relación 

In [23]:
romantic = [
     {'id_romantic': 1, 'romantic': 'no'}, 
     {'id_romantic': 2, 'romantic': 'yes'}
]

insertDataToSQL(romantic, 'p_romantic')

(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "p_romantic_pkey"
DETAIL:  Key (id_romantic)=(1) already exists.

[SQL: INSERT INTO p_romantic (id_romantic, romantic) VALUES (%(id_romantic)s, %(romantic)s)]
[parameters: ({'id_romantic': 1, 'romantic': 'no'}, {'id_romantic': 2, 'romantic': 'yes'})]
(Background on this error at: https://sqlalche.me/e/14/gkpj)


#### Insertar escuela

In [12]:
school = [
     {'id_school': 1, 'school': 'GP'}, 
     {'id_school': 2, 'school': 'MS'}
]

insertDataToSQL(school, 'p_school')

(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "p_school_pkey"
DETAIL:  Key (id_school)=(1) already exists.

[SQL: INSERT INTO p_school (id_school, school) VALUES (%(id_school)s, %(school)s)]
[parameters: ({'id_school': 1, 'school': 'GP'}, {'id_school': 2, 'school': 'MS'})]
(Background on this error at: https://sqlalche.me/e/14/gkpj)


#### Insertar edad

In [18]:
age = [
     {'id_age': 1, 'age': '15'}, 
     {'id_age': 2, 'age': '16'},
     {'id_age': 3, 'age': '17'},
     {'id_age': 4, 'age': '18'},
     {'id_age': 5, 'age': '19'},
     {'id_age': 6, 'age': '20'},
     {'id_age': 7, 'age': '21'},
     {'id_age': 8, 'age': '22'},
]

insertDataToSQL(age, 'p_age')

(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "p_age_pkey"
DETAIL:  Key (id_age)=(1) already exists.

[SQL: INSERT INTO p_age (id_age, age) VALUES (%(id_age)s, %(age)s)]
[parameters: ({'id_age': 1, 'age': '15'}, {'id_age': 2, 'age': '16'}, {'id_age': 3, 'age': '17'}, {'id_age': 4, 'age': '18'}, {'id_age': 5, 'age': '19'}, {'id_age': 6, 'age': '20'}, {'id_age': 7, 'age': '21'}, {'id_age': 8, 'age': '22'})]
(Background on this error at: https://sqlalche.me/e/14/gkpj)


In [22]:
archivo = "D:\Clips\T.xlsx"
df = pd.read_excel(archivo, sheet_name='Calificaciones')


#### Insertamos datos de las calificaciones

In [ ]:
def insertDataToSQL(data_dict, p_grades):
     postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""    
     df_data = df
     try:
          response = df_data.to_sql(p_grades, postgres_driver, index=False, if_exists='append')
          print(f'Se han insertado {response} nuevos registros.' )
     except Exception as ex:
          print(ex)

In [27]:
clases = ["matemáticas", "idioma"]
clase_aleatoria = random.randint(0,1)
print("La clase seleccionada es:", clases[clase_aleatoria])

La clase seleccionada es: idioma


In [29]:
cantidad_calificaciones = np.random.randint(1, 1200)
data_calificaciones = []

for calific in range(cantidad_calificaciones):
    nueva_calificacion = {
            'id_grades': calific,
            'id_school': random.sample(school, 1)[0]['id_school'],
            'id_sex': random.sample(sexo, 1)[0]['id_sex'],
            'id_age': random.sample(age, 1)[0]['id_age'],
            'id_job': random.sample(t_trabajo, 1)[0]['id_job'],
            'id_guardian': random.sample(guardian, 1)[0]['id_guardian'],
            'id_internet': random.sample(internet, 1)[0]['id_internet'],
            'id_romantic': random.sample(romantic, 1)[0]['id_romantic'],
            'g1':round(random.uniform(0, 20), 2),
            'g2': round(random.uniform(0, 20), 2),  
            'g3': round(random.uniform(0, 20), 2),   
            'subject': random.randint(0,1)      
        }
    
    data_calificaciones.append(nueva_calificacion)
   
insertDataToSQL(data_calificaciones, 'p_grades')

Se han insertado 936 nuevos registros.
